# **Метрические методы классификации**

*1 Методы ближайших соседей

В этой части работы вы:

• научитесь готовить данные к построению модели (предобработка, или препро-
цессинг данных);

• познакомитесь с методами ближайших соседей для задач классификации и
регрессии, реализованными в библиотеке scikit-learn ;
• научитесь оценивать качество модели с помощью отложенной выборки.*

Указания к выполнению
1. Подключитесь к одному из наборов данных на Kaggle. Разберитесь в том, как устроен ваш датасет и какова постановка задачи.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10, 8)});

In [ ]:
 df = pd.read_csv('../input/bank-marketing-dataset/bank.csv')
df.head()

In [ ]:
 df.info()

In [ ]:
df.describe()

Это классический набор данных маркетингового банка, изначально загруженный в репозиторий машинного обучения UCI. Набор данных дает нам информацию о маркетинговой кампании финансового учреждения, которую нам нужно будет проанализировать, чтобы найти способы поиска будущих стратегий для улучшения будущих маркетинговых компаний для банка.

Целевая переменная deposit-принимает значения да или нет.

Задача классификации: выяснить, что клиент возьмет депозит.

2. Извлеките целевой признак (target). Какая из задач обучения с учителем рассматривается – классификация или регрессия?

In [ ]:
df['deposit']

Имеем конечное количество "yes" и "no". отсюда следует, что из задач обучения с учителем рассматривается – классификация.

3. Каково распределение значений target-переменной? Постройте подходящую визуализацию. Прокомментируйте результат.

In [ ]:
df['deposit'].value_counts().plot(kind = 'bar', color = 'green')

Классы почти сбалансированы, так как количество "no"совсем немного превышает"yes".

4. Проведите необходимую предобработку данных (preprocessing). Для построения моделей с помощью метрических методов все признаки должны быть закодированы числами. Полезными будут следующие методы библиотеки Pandas:

• map() –– для перекодировки категориальной переменной числовыми метками;

• get_dummies() –– для создания нескольких бинарных признаков на основе категориального.
Также может потребоваться масштабирование данных (scaling). Воспользуйтесь
классом StandardScaler библиотеки Scikit-learn.

In [ ]:
df1 = pd.read_csv('/kaggle/input/bank-marketing-dataset/bank.csv')


In [ ]:
df1.info()

In [ ]:

df1 = pd.get_dummies(df, columns=['deposit','job', 'education', 'marital', 'contact', 'poutcome', 'month', 'loan', 'housing', 'default'])
df1['deposit']=df['deposit'].map({'no':0,'yes':1})

In [ ]:
df2 = df1.drop(['deposit'], axis=1)

In [ ]:
df2.head()

In [ ]:
df1.head().T

5. Разбейте набор данных на обучающую и валидационную (тестовую) выборки с помощью метода train_test_split .

In [ ]:
from sklearn.model_selection import train_test_split

# Создание X, y
# X --- вся таблица без таргета
# y --- таргет (целевая переменная)
X=df1.drop('deposit', axis=1)
y =df1['deposit']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=12)

In [ ]:
X.info()

6. Обучите алгоритм классификации kNeighborsClassifier или регрессии KNeighborsRegressor . Оцените качество каждой модели на валидационной выборке с помощью
• accuracy_score для классификации;
• mean_squared_error для регрессии.
Сравните результаты и сделайте выводы.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X, y)

In [ ]:
neigh.score(X_valid, y_valid)

In [ ]:
neigh.fit(X_train,y_train)

In [ ]:
# from sklearn.neighbors import KNeighborsRegressor
# neigh = KNeighborsRegressor(n_neighbors=5)
# neigh.fit(X, y)

In [ ]:
# neigh.score(X_valid, y_valid)

In [ ]:
y_pred = neigh.predict(X_valid)

In [ ]:
from sklearn.metrics import accuracy_score

neigh.score(X_valid, y_valid) 

In [ ]:
print('Качество модели:', accuracy_score(y_pred, y_valid)) 

Мы получаем практически идентичный результат, следственно это является  лучшим результатом.

2 Настройка оптимального числа ближайших соседей в методе kNN

В данной части работы вы научитесь настраивать параметр n_neighbors алгоритма kNN с помощью перекрёстной проверки (кросс-валидации).

Указания к выполнению

1. Создайте генератор разбиений, который перемешивает выборку перед созданием блоков ( shuffle=True ). Число блоков n_splits равно 5. Задайте также параметр random_state для воспроизводимости результатов. 
Например: kf = KFold(n_splits=5, shuffle=True, random_state=42)

Найдите показатель качества модели kNN на кросс-валидации. Подумайте, приемлемо ли использование вашей меры (метрики) качества в данной задаче? При необходимости пересчитайте качество с помощью другой метрики из списка.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=42) # n_splits играет роль K
scores = cross_val_score(neigh, X, y, cv=kf, scoring='accuracy')
print('Массив значений метрики:', scores)
print('Средняя метрика на кросс-валидации:', np.mean(scores))

2. Осуществите кросс-валидацию модели при числе соседей k ∈ [1;50]. Используйте GridSearchCV . При каком k качество получилось наилучшим? Чему равна эта оценка качества? Постройте график значений метрики в зависимости от k
( matplotlib.pyplot.plot() ).

In [ ]:
from sklearn.model_selection import GridSearchCV
neigh_params={'n_neighbors': np.arange(1, 50)} 
neigh_grid = GridSearchCV(neigh, neigh_params, cv=kf, scoring='accuracy') # кросс-валидация по 5 блокам
neigh_grid.fit(X_train, y_train)

In [ ]:
# Смотрим лучшие значения параметров
print(neigh_grid.best_params_)

# Лучшая модель
print(neigh_grid.best_estimator_)

In [ ]:
pd.DataFrame(neigh_grid.cv_results_).T

In [ ]:
import matplotlib.pyplot as plt
results_df = pd.DataFrame(neigh_grid.cv_results_)
plt.plot(results_df['param_n_neighbors'], results_df['mean_test_score'])

# Подписываем оси и график
plt.xlabel('Number of neighbors')
plt.ylabel('Test accuracy')
plt.title('Validation curve')
plt.show()

3 Выбор метрики в методе kNN

Главным параметром любого метрического алгоритма является функция расстояния (метрика), используемая для измерения сходства между объектами. Можно использовать стандартный вариант (например, евклидову метрику), но гораздо более

эффективным вариантом является подбор метрики под конкретную задачу. Один из подходов –– использование той же евклидовой метрики, но с весами: каждой координате ставится в соответствие определенный коэффициент; чем он больше, тем выше вклад признака в итоговое расстояние. Веса настраиваются с целью оптимизации качества на отложенной выборке.
Другой одход, о котором и пойдет речь в данной части работы –– выбор метрики из некоторого класса метрик. Мы возьмем за основу метрику Минковского

Параметром метрики Минковского является число p, которое мы и будем настраивать.

In [ ]:
p_params = {"p": np.linspace(1,10,200)}
neigh = KNeighborsClassifier(n_neighbors=5, weights = "distance", n_jobs = -1)
cv = GridSearchCV(neigh, p_params, cv = kf, scoring="f1", verbose = 100)
cv.fit(X,y)

In [ ]:
pd.DataFrame(neigh_grid.cv_results_).T

In [ ]:
neigh_grid.best_score_

In [ ]:
neigh_grid.best_params_

4 Другие метрические методы

Поэкспериментируйте с другими метрическими методами для задач регрессии и клас-
сификации, представленными в библиотеке Scikit-learn:

• RadiusNeighborsClassifier ;
• RadiusNeighborsRegressor ;
• NearestCentroid .

In [ ]:
from sklearn.neighbors import RadiusNeighborsRegressor
neigh = RadiusNeighborsRegressor(radius=1.0)
neigh.fit(X, y)
neigh.fit(X_train, y_train)
neigh.score(X_valid, y_valid)

In [ ]:
from sklearn.neighbors import NearestCentroid
clf = NearestCentroid()
clf.fit(X, y)
clf.fit(X_train, y_train)
clf.score(X_valid, y_valid)